In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score, accuracy_score

2025-10-08 20:28:33.232867: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-08 20:28:33.240804: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-08 20:28:33.262808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759935513.299184   49106 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759935513.309804   49106 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759935513.339313   49106 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [ ]:
class MilitaryRoboticSystemMaintenance:
    def __init__(self):
        self.model = None
        self.scaler = StandardScaler()
        self.feature_names = None

    def generate_synthetic_data(self, n_samples=1000):
        df = pd.read_csv('smart_manufacturing_temperature_regulation.csv')
        iot_df = pd.read_csv('industrial_fault_detection_data.csv')
        robot_df = pd.read_csv('industrial_robot_control.csv')
        np.random.seed(42)
        current_temperature = df['Current Temperature (°C)'].values
        setpoint_temperature = df['Setpoint Temperature (°C)'].values  
        ambient_temperature = df['Ambient Temperature (°C)'].values
        system_vibration = iot_df['Vibration (mm/s)'].values
        hydraulic_pressure = iot_df['Pressure (bar)'].values
        operational_hours = robot_df['task_duration'].values
        current_draw = robot_df['energy_consumption'].values[:n_samples]

        data = {
        'system_temperature': current_temperature,
        'power_unit_temperature': setpoint_temperature,
        'ambient_temperature': ambient_temperature,
        'system_vibration': system_vibration,
        'hydraulic_pressure': hydraulic_pressure,
        'current_draw': current_draw,
        'operational_hours': operational_hours,
        'previous_failures' : robot_df['error_rate'].values * 10,
        }
        failure_probability = (
            0.3 * (data['system_temperature'] > 70).astype(int) +
            0.4 * (data['power_unit_temperature'] > 80).astype(int) +
            0.4 * (data['system_vibration'] > 2.0).astype(int) +
            0.2 * (data['operational_hours'] > 4000).astype(int) +
            0.3 * (data['previous_failures'] > 2).astype(int)
        ) / 1.6
        data['is_failure'] = np.random.binomial(1, failure_probability)
        df = pd.DataFrame(data)
        df['timestamp'] = pd.date_range(start='2024-01-01',
        periods=n_samples, freq='h')
        return df
    def preprocess_data(self, data):
        self.feature_names = [col for col in data.columns if col not in ['is_failure', 'timestamp']]
        X = data[self.feature_names]
        y = data['is_failure']
        X_scaled = self.scaler.fit_transform(X)
        return train_test_split(X_scaled, y, test_size=0.3, random_state=42)
    def train_model(self, X_train, y_train):
        self.model = RandomForestClassifier(
        n_estimators=20,
        max_depth=5,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42
        )
        self.model.fit(X_train, y_train)
    def evaluate_model(self, X_test, y_test):
        y_pred = self.model.predict(X_test)
        print("Classification Report:")
        print(classification_report(y_test, y_pred))
        cm = confusion_matrix(y_test,
        y_pred)
        plt.figure(figsize=(3, 3))
        sns.heatmap(cm, annot=True, fmt='d',
        cmap='Blues')
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()
        # ROC Curve
        y_pred_proba = self.model.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test,
        y_pred_proba)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(4, 4))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.show()
    def verify_with_gru(self, X_train, X_test, y_train, y_test, epochs=30, batch_size=32):
        X_test_rf = X_test
        # 🔹 Reshape for GRU (3D)
        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
        
        # Build GRU model
        model = Sequential([
            GRU(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False),
            Dropout(0.3),
            Dense(32, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        
        # Compile and train
        model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1)
        
        # Predict on test set
        y_pred_prob = model.predict(X_test)
        y_pred = (y_pred_prob > 0.5).astype(int)
        
        # Evaluate GRU
        gru_f1 = f1_score(y_test, y_pred)
        gru_acc = accuracy_score(y_test, y_pred)
        print(f"\n✅ GRU Model Evaluation:")
        print(f"Accuracy: {gru_acc:.4f}")
        print(f"F1 Score: {gru_f1:.4f}")
        
        rf_pred = self.model.predict(X_test_rf)
        rf_f1 = f1_score(y_test, rf_pred)
        rf_acc = accuracy_score(y_test, rf_pred)
        
        print(f"\n📊 Random Forest vs GRU Comparison:")
        print(f"Random Forest - Accuracy: {rf_acc:.4f}, F1: {rf_f1:.4f}")
        print(f"GRU Model     - Accuracy: {gru_acc:.4f}, F1: {gru_f1:.4f}")
        
        return {
            'random_forest': {'accuracy': rf_acc, 'f1': rf_f1},
            'gru': {'accuracy': gru_acc, 'f1': gru_f1}
        }

    def analyze_feature_importance(self):
        importances = self.model.feature_importances_
        indices = np.argsort(importances)[::-1]
        plt.figure(figsize=(8, 4))
        plt.title('Feature Importance for Failure Prediction')
        plt.bar(range(len(importances)), importances[indices])
        plt.xticks(range(len(importances)), [self.feature_names[i] for i in indices], rotation=45)
        plt.tight_layout()
        plt.show()
    def correlation_heatmap(self, data):
        correlation_matrix = data.corr()
        plt.figure(figsize=(8, 6))
        sns.heatmap(correlation_matrix,
        annot=True, fmt='.2f', cmap='coolwarm')
        plt.title('Feature Correlation Heatmap')
        plt.show()

    def feature_distribution(self, data):
        features = [
        'system_temperature',
        'power_unit_temperature',
        'ambient_temperature',
        'system_vibration',
        'hydraulic_pressure',
        'current_draw',
        'operational_hours',
        'previous_failures'
        ]
        plt.figure(figsize=(15, 10))
        for i, feature in enumerate(features):
            plt.subplot(4, 4, i + 1)
            sns.histplot(data, x=feature,
            hue='is_failure', kde=True, bins=30,
            palette='coolwarm')
            plt.title(f'Distribution of {feature}')
        plt.tight_layout()
        plt.show()
    def time_series_plot(self, data):
        time_grouped = data.groupby(data['timestamp'].dt.date)['is_failure'].mean()
        plt.figure()
        plt.plot(time_grouped.index,time_grouped.values, marker='o',color='teal')
        plt.title('Average Failure ProbabilityOver Time')
        plt.xlabel('Date')
        plt.ylabel('Failure Probability')
        plt.xticks(rotation=45)
        plt.grid()
        plt.show()
    def predict_maintenance_needs(self,current_data):
        scaled_data = self.scaler.transform(current_data[self.feature_names])
        failure_prob = self.model.predict_proba(scaled_data)[:, 1]
        predictions = pd.DataFrame({
            'Failure_Probability': failure_prob,
            'Risk_Level': pd.cut(failure_prob,
        bins=[0, 0.3, 0.6, 1],
        labels=['Low', 'Medium', 'High'])})
        return predictions